In [1]:
!nvidia-smi  # Confirm GPU is active

Tue Apr 29 16:06:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!pip install --upgrade torch torchvision torchaudio transformers sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 972.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

# Evidence-Based Policy Scoring with Open LLMs (Zero-Shot)

This notebook lets you score policy documents using an open-source LLM and a rubric-based prompt. It uses Mistral-7B via Hugging Face. Steps include:
* Load Mistral-7B-Instruct via Hugging Face
* Accept .txt policy files in a folder
* Prompt the model using the evidence-based policy (EBP) rubric
* Parse the response and output a .csv with scores and justifications for them

> Go to Runtime > Change runtime type > GPU in Colab if not doing it locally.


In [9]:
!pip install -q transformers accelerate bitsandbytes sentencepiece
!pip install -q unstructured
!pip install bitsandbytes --prefer-binary --no-cache-dir --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
import re

# Handler for bad model text/need to only parse valid json blocks
def extract_first_json(text):
    matches = re.findall(r"\{(?:[^{}]|(?:\{.*?\}))*\}", text, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue
    print("No valid JSON block found.")
    return None

### Loading in models
Note:  Mistral is a huge model and will take a long time to cache initially. It will also take a long time to process docs through it. A smaller model, like Llama-3b, is easier to use locally via CPU. Use a GPU (e.g., Colab) to run Mistral.

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
rubric_prompt = """
Use the rubric below to score the policy document on five dimensions of evidence-based policymaking.
Each dimension should receive a score from 0 to 3, where:

0 = No evidence of the dimension
1 = Minimal or implicit evidence
2 = Moderate or partial evidence
3 = Clear, strong, and explicit evidence

### Dimensions:

1. Use of Empirical Research
- 0: No references to empirical evidence or data
- 1: Vague or anecdotal references (e.g., “studies show”)
- 2: Clear empirical support, but limited sourcing
- 3: Multiple, clearly cited, high-quality sources (e.g., peer-reviewed, systematic reviews)

2. Formal Evidence-Gathering Process
- 0: No structured data gathering
- 1: Informal or anecdotal input
- 2: Basic assessments (e.g., internal reports, cost estimates)
- 3: Formal tools (e.g., RCTs, modeling, pilot programs)

3. Transparency and Accessibility
- 0: No documentation or rationale
- 1: Minimal or internal-only documentation
- 2: Public access with basic explanation
- 3: Fully open access, replicable, with detailed methods

4. Expert and Stakeholder Input
- 0: No input from external parties
- 1: Informal or minimal consultation
- 2: Some formal consultation or review by stakeholders
- 3: Broad engagement with named experts or stakeholders

5. Evaluation and Iteration
- 0: No plan or evidence of evaluation
- 1: A vague or one-time evaluation mention
- 2: Evaluation included with some details
- 3: Robust plan or evidence of ongoing evaluation and iteration

### Note:
You will be asked to return a JSON object based on a policy document. Each dimension must include:
- A `score` (0–3)
- A `justification` with a full sentence explaining the score

Do not reuse this rubric text or copy examples into your answer.
"""

In [7]:
import os

batch_folder = "/content/drive/MyDrive/batch_01"

# List .txt files to make sure paths are talking to each other
txt_files = [f for f in os.listdir(batch_folder) if f.endswith('.txt')]

print(f"Found {len(txt_files)} text files")
print(txt_files[:5])

Found 2000 text files
['7-5700.txt', '94-803.txt', '94-834.txt', '95-1013.txt', '94-166.txt']


In [8]:
os.getcwd()

'/content'

In [9]:
from pathlib import Path
import os
import pandas as pd
import json
import re
from transformers import pipeline

# Set batch folder
batch_folder = "/content/drive/MyDrive/batch_01"
txt_folder = Path(batch_folder)
results = []

# Define prompt rubric (keep the existing rubric_prompt from previous cell)
for filename in os.listdir(txt_folder):
    if not filename.endswith(".txt"):
        continue
    print(f"Processing {filename}")
    with open(txt_folder / filename, "r", encoding="utf-8") as f:
        doc_text = f.read()

    # Final prompt structure with no static JSON example, only dynamic instruction
    full_prompt = (
        "You are a policy analyst evaluating how evidence-based a policy document is.\n"
        "Below is a policy document. Read it carefully and then apply the rubric that follows.\n\n"
        + doc_text[:3500] + "\n\n"
        + "Now use this rubric to evaluate:\n"
        + rubric_prompt + "\n\n"
        + "Return only a JSON object. Each key should be a rubric dimension. Each value should be an object "
          "with a 'score' (0–3) and a 'justification' — a sentence explaining why that score was given based on the text of the document.\n"
        + "Do not reuse examples or include placeholder text like '...'. Be specific, original, and concise in your justifications."
    )

    response = llm(full_prompt)[0]["generated_text"]
    print(f"Raw model output for {filename} (first 1000 chars):\n", response[:1000])

    parsed = extract_first_json(response)
    if not parsed:
        print(f"Could not extract valid JSON for {filename}")
        continue

    flat = {"filename": filename}
    for k, v in parsed.items():
        if isinstance(v, dict):
            flat[f"{k} Score"] = v.get("score")
            flat[f"{k} Justification"] = v.get("justification")
        else:
            flat[f"{k} Score"] = v
            flat[f"{k} Justification"] = ""
    results.append(flat)

if results:
    df = pd.DataFrame(results)
    df.to_csv("evidence_scores.csv", index=False)
    print("CSV saved as 'evidence_scores.csv'")
else:
    print("No valid outputs parsed.")

Processing 7-5700.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Raw model output for 7-5700.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.


 
Introducing a Public Advocate into the Foreign Intelligence Surveillance Act’s Courts: Select Legal Issues 
Andrew Nolan Legislative Attorney 
Richard M. Thompson II Legislative Attorney 
Vivian S. Chu Legislative Attorney 
October 25, 2013 
Congressional Research Service 
7-5700 
www.crs.gov 

CRS Report for CongressP
repared for Members and Committees of Congress  

Introducing a Public Advocate into the FISA's Courts: Relevant Legal Issues 
 
Summary 
Recent revelations about the size and scope of government foreign surveillance efforts have prompted some to criticize the level of scrutiny that the courts – established under the Foreign Intelligence Surveillance Act of 1978 (FISA) – currently provide with respect to the government’s applications to engage in such sur

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Raw model output for 94-803.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.




 
 
 
Social Security: Cost-of-Living Adjustments 
Updated January 15, 2025 
Congressional Research Service 
https://crsreports.congress.gov 
94-803 


Social Security: Cost-of-Living Adjustments 
 
Congressional Research Service 
Summary 
To compensate for the effects of inflation, Social Security recipients usually receive an annual cost-of-living adjustment (COLA). According to parameters outlined in the Social Security Act (42 U.S.C. §415(i)), a 2.5% COLA is payable starting in January 2025. For a retired worker, the 2024 COLA will raise the estimated average monthly benefit amount received in January 2025 by $49 from $1,927 to $1,976 (after final rounding down to the nearest dollar). 
Social Security COLAs are based on changes in the Consumer Price Index for Urban 

...

Raw model output for 97-765A.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.




Order Code 97-765 A
Updated August 29, 2008
The Buy American Act: Requiring
Government Procurements to Come from
Domestic Sources
John R. Luckey
Legislative Attorney
 American Law Division
Summary
The Buy American Act is the major domestic preference statute governing
procurement by the federal government.  Essentially it attempts to protect domestic laborby providing a preference for American goods in government purchases.  It was enactedin 1933 and has only been substantively amended four times in the succeeding years.In determining what are American goods, the place of mining, production, ormanufacture is controlling.  The nationality of the contractor is not considered whendetermining if a product is of domestic origin.
In the 110th Congress a new reporting require

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Raw model output for 97-780.txt (first 1000 chars):
 You are a policy analyst evaluating how evidence-based a policy document is.
Below is a policy document. Read it carefully and then apply the rubric that follows.


The Speaker of the House: House Officer, Party Leader, and Representative
May 16, 2017
            (97-780)
          Jump to Main Text of Report



Contents

Introduction
Selection of the Speaker
The Speaker as Leader of the House
The Speaker as Party Leader
The Speaker as a Member of the House


Appendixes

Appendix A. Speakers of the House of Representatives, 1789-2017
Appendix B. Select Bibliography



Summary
The Speaker of the House of Representatives is widely viewed as symbolizing the power and authority of the House. The Speaker's most prominent role is that of presiding officer of the House. In this capacity, the Speaker is empowered by House rules to administer proceedings on the House floor, including recognition of Members to speak on the floor or make motion

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [10]:
df = pd.DataFrame(results)

In [11]:
df.to_csv("evidence_scores.csv", index=False)